In [1]:
import subprocess
import shlex
import os
import glob
import re

from subprocess import PIPE
from pathlib import Path

In [2]:
TIMEOUT_PATTERN = re.compile(r"=>> PBS: job killed: walltime (?:\d+) exceeded limit 259200\n")

In [3]:
output = Path("results/limit").resolve(strict=False)
output.mkdir(parents=True, exist_ok=True)
    
glob_pathname = f"{glob.escape(output)}.e*.*"
    
successful = []
erroneous = []
    
for path in glob.iglob(glob_pathname):
    with open(path) as file:
        content = file.read()
        
        if content == '' or TIMEOUT_PATTERN.fullmatch(content):
            successful.append(path)
        else:
            erroneous.append(path)

for error_path in erroneous:
    Path('o'.join(error_path.rsplit('e', 1))).unlink()
    Path(error_path).unlink()
        
repeats = 10 - len(successful)

if repeats >= 1:
    print(f"{repeats} x {output}")

    repeats = max(repeats, 2)

    raise "hi"

    speciations = " ".join(str(10.0**(-e)) for e in range(0, 20))

    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={32}gb
    #PBS -J {0}-{max(repeats - 1, 0)}
    #PBS -lwalltime={72}:00:00

    for SPECIATION in {speciations}
    do
        $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
            speciation: '$SPECIATION',
            sample: {0.025},
            seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

            algorithm: SkippingGillespie(),

            scenario: SpatiallyExplicit(
                habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
                dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
            ),

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                    reporters: [Counter(), Execution()],
                ),
            ],
        )'
    done
    """

    cwd = os.getcwd()
    os.chdir(output.parent)

    print(subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip())

    os.chdir(cwd)